##### Copyright 2024 Google LLC.

#### **Accelerator:** TPU VM v3-8

In [22]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/keras_inference"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Get started with Gemma using KerasNLP

This tutorial shows you how to get started with Gemma using [KerasNLP](https://keras.io/keras_nlp/). Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models. KerasNLP is a collection of natural language processing (NLP) models implemented in [Keras](https://keras.io/) and runnable on JAX, PyTorch, and TensorFlow.

In this tutorial, you'll use Gemma to generate text responses to several prompts. If you're new to Keras, you might want to read [Getting started with Keras](https://keras.io/getting_started/) before you begin, but you don't have to. You'll learn more about Keras as you work through this tutorial.

## Setup

### Gemma setup

To complete this tutorial, you will first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

Gemma models are hosted by Kaggle. To use Gemma, request access on Kaggle:

- Sign in or register at [kaggle.com](https://www.kaggle.com)
- Open the [Gemma 2 model card](https://www.kaggle.com/models/google/gemma-2) and select _"Request Access"_
- Complete the consent form and accept the terms and conditions


### Install dependencies

Install Keras and KerasNLP.

In [2]:
# # Install Keras 3 last. See https://keras.io/getting_started/ for more details.
# !pip install -q -U keras-nlp
# !pip install -q -U "keras>=3"

In [3]:
!pip install -q -U keras-nlp
# Work around an import error with tensorflow-hub. The library is not used.
!pip install -q -U tensorflow-hub
# Install tensorflow-cpu so tensorflow does not attempt to access the TPU.
!pip install -q -U tensorflow-cpu tensorflow-text
# Install keras 3 last. See https://keras.io/getting_started for details.
!pip install -q -U keras


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.17.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import jax

jax.devices()

E0000 00:00:1729003078.404204      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
E1015 14:37:58.443535641     117 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-10-15T14:37:58.443514262+00:00", grpc_status:2}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [6]:
jax.__version__

'0.4.23'

In [7]:
import keras

keras.__version__

2024-10-15 14:38:36.721395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 14:38:36.743891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 14:38:36.750782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'3.6.0'

In [8]:
import time

start_time = time.perf_counter()

print("Hello, Gemma 2 Demo!")

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Execution Time: {execution_time:.3f} ms")

Hello, Gemma 2 Demo!
Execution Time: 0.129 ms


### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. [Keras 3](https://keras.io/keras_3) lets you choose the backend: TensorFlow, JAX, or PyTorch. All three will work for this tutorial.

In [9]:
import os

# os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation
# overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

In [10]:
# import gc
# gc.collect()

### Import packages

Import Keras and KerasNLP.

In [11]:
import keras
import keras_nlp

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create a model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/). In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

In [12]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


The `GemmaCausalLM.from_preset()` function instantiates the model from a preset architecture and weights. In the code above, the string `"gemma2_2b_en"` specifies the preset the Gemma 2 2B model with 2 billion parameters. Gemma models with [7B, 9B, and 27B parameters](/gemma/docs/get_started#models-list) are also available. You can find the code strings for Gemma models in their **Model Variation** listings on [Kaggle](https://www.kaggle.com/models/google/gemma).


Note: To run the larger models in Colab, you need access to the premium GPUs available in paid plans. Alternatively, you can perform inferences using Kaggle notebooks or Google Cloud projects.


Use `summary` to get more info about the model:

In [13]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

As you can see from the summary, the model has 2.6 billion trainable parameters.

Note: For purposes of naming the model ("2B"), the embedding layer is not counted against the number of parameters.

## Generate text

Now it's time to generate some text! The model has a `generate` method that generates text based on a prompt. The optional `max_length` argument specifies the maximum length of the generated sequence.

**Demo-1:** Try it out with the prompt `"what is keras in 3 bullet points?"`.

In [14]:
prompt = """<start_of_turn>user
what is keras in 3 bullet points?<end_of_turn>
<start_of_turn>model
"""
start_time = time.perf_counter()

output = gemma_lm.generate(prompt, max_length=256)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(output)

I0000 00:00:1729003497.604926    1091 service.cc:146] XLA service 0x7a93e460f7f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729003497.604996    1091 service.cc:154]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1729003497.627111    1091 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Inference Time: 53269.405 ms
<start_of_turn>user
what is keras in 3 bullet points?<end_of_turn>
<start_of_turn>model
Here's a breakdown of Keras in 3 bullet points:

* **High-Level API:** Keras provides a user-friendly interface for building and training neural networks, simplifying the process of working with complex models.
* **Flexible and Modular:** Keras is built on top of TensorFlow, offering a modular design that allows you to easily integrate with other TensorFlow components or use it independently.
* **Rapid Prototyping:** Keras excels at rapid prototyping, enabling you to quickly experiment with different architectures and hyperparameters to find optimal solutions for your machine learning tasks. 
<end_of_turn>


**Demo-1 Imitation:** Try it out with the prompt `"Give me some ways to Interlaken from Zurich?"`.

In [15]:
prompt = """<start_of_turn>user
Give me some ways to Interlaken from Zurich?<end_of_turn>
<start_of_turn>model
"""

start_time = time.perf_counter()

output = gemma_lm.generate(prompt, max_length=512)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(output)

Inference Time: 125113.306 ms
<start_of_turn>user
Give me some ways to Interlaken from Zurich?<end_of_turn>
<start_of_turn>model
Here are some ways to get from Zurich to Interlaken:

**By Train:**

* **Direct Train:** The most convenient option is taking a direct train from Zurich HB (main station) to Interlaken Ost. This journey takes around 2 hours and 15 minutes. You can find tickets on the Swiss Federal Railways (SBB) website or at the station.
* **Change in Lucerne:** You can also take a train from Zurich HB to Lucerne, then change to a train to Interlaken Ost. This option might be slightly cheaper but adds a bit more travel time.

**By Bus:**

* **Bus:** Several bus companies offer routes from Zurich to Interlaken. This option is generally more affordable than the train but can take longer, especially during peak season. You can find bus schedules and book tickets on websites like FlixBus or PostAuto.

**By Car:**

* **Driving:** If you prefer to drive, it takes around 2 hours an

**Demo-1 Imitation (Chinese):** Try it out with the prompt `"新疆伊犁的最佳旅游时节和攻略？"`.

In [19]:
prompt = """<start_of_turn>user
新疆伊犁的最佳旅游时节和攻略？<end_of_turn>
<start_of_turn>model
"""

start_time = time.perf_counter()

output = gemma_lm.generate(prompt, max_length=1024)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(output)

Inference Time: 357756.787 ms
<start_of_turn>user
新疆伊犁的最佳旅游时节和攻略？<end_of_turn>
<start_of_turn>model
## 新疆伊犁最佳旅游时节和攻略

**最佳旅游时节:**

* **春季 (4月-5月):**  天气温暖，景色宜人，适合观赏花海和草原。
* **夏季 (6月-8月):**  天气炎热，适合体验伊犁的自然风光和文化活动。
* **秋季 (9月-10月):**  天气凉爽，适合观赏秋色和体验伊犁的传统节日。
* **冬季 (11月-3月):**  天气寒冷，适合体验雪景和冰雪活动。

**具体推荐:**

* **春季:**  
    * **推荐景点:**  
        * **哈密:**  游览哈密古城，感受古老的文化和历史。
        * **库尔勒:**  参观库尔勒博物馆，了解伊犁的文化和历史。
        * **阿克苏:**  体验阿克苏的特色美食和文化活动。
    * **活动:**  
        * **观赏花海:**  春季伊犁的草原和山区都呈现出美丽的景色，适合观赏各种花卉。
        * **体验草原文化:**  参加草原文化体验活动，感受伊犁的传统生活方式。
* **夏季:**  
    * **推荐景点:**  
        * **库尔勒:**  参观库尔勒的自然景观，例如库尔勒湖和库尔勒山。
        * **哈密:**  游览哈密的自然景观，例如哈密山和哈密湖。
        * **阿克苏:**  体验阿克苏的特色美食和文化活动。
    * **活动:**  
        * **体验草原活动:**  参加草原骑马、野营等活动，感受伊犁的自然风光。
        * **观赏自然景观:**  夏季伊犁的自然景观更加壮观，适合观赏草原和山脉。
* **秋季:**  
    * **推荐景点:**  
        * **哈密:**  游览哈密的自然景观，例如哈密山和哈密湖。
        * **库尔勒:**  参观库尔勒的自然景观，例如库尔勒湖和库尔勒山。
        * **阿克苏:**  体验阿克苏的特色美食和文化活动。
    * **活动:**  
     

**Demo-2:** Try calling `generate` again with a different prompt.

In [17]:
start_time = time.perf_counter()

results = gemma_lm.generate("list, in less than 100 words, 5 things you can help me do?", max_length=128)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 34421.121 ms
list, in less than 100 words, 5 things you can help me do?

1. **Write a poem about a lost cat.**
2. **Create a list of fun activities for a rainy day.**
3. **Generate a funny joke.**
4. **Help me brainstorm ideas for a birthday party.**
5. **Explain a complex scientific concept in simple terms.** 


<end_of_turn>


**Demo-2 Imitation**

In [18]:
start_time = time.perf_counter()

results = gemma_lm.generate("table, in less than 100 words, 3 most impressive final matches of the World Cup?", max_length=128)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 31361.442 ms
table, in less than 100 words, 3 most impressive final matches of the World Cup?

Here are three of the most impressive final matches of the World Cup:

1. **1982: West Germany vs. Italy:** A tense, tactical battle that saw West Germany emerge victorious with a 3-1 win.  
2. **2006: Italy vs. France:** A thrilling match that went down to the wire, with Italy ultimately winning 5-3 on penalties after a 1-1 draw.
3. **2018: France vs. Croatia


**Demo-2 Imitation (Japanese)**

In [20]:
start_time = time.perf_counter()

results = gemma_lm.generate("リスト, 250語以内で, 桜の季節におすすめの青森のお花見スポット?", max_length=256)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 71855.625 ms
リスト, 250語以内で, 桜の季節におすすめの青森のお花見スポット?

## 桜の季節におすすめの青森のお花見スポット

青森県は、桜の名所としても知られています。美しい桜並木が点在するスポットで、春の訪れを満喫できます。ここでは、おすすめの桜の季節におすすめのスポットを250語以内でご紹介します。

**1. 青森県立美術館:**  美術館の庭園には、約100本の桜が咲き誇ります。美術館の雰囲気を楽しみながら、桜を眺められます。

**2. 青森県庁舎:**  県庁舎の庭園には、約100本の桜が咲き誇ります。県庁舎の美しい建築物と桜の景色が、まさに芸術作品のような光景です。

**3. 青森県立博物館:**  博物館の庭園には、約100本の桜が咲き誇ります。博物館の展示物と桜の景色を楽しみながら、歴史に触れることができます。

**4. 青森市公園:**  市内にある公園には、約100本の桜が咲き誇ります。公園の緑豊かな自然と桜の景色が、心身ともにリフレッシュ


#### <font color=red>**Attention:** The answers are less relevant to the question</font> 
* <font color=blue>Consider a **relevance** **model** for **ranking** to pick the most relevant answers from proposals</font>

If you're running on JAX or TensorFlow backends, you'll notice that the second `generate` call returns nearly instantly. This is because each call to `generate` for a given batch size and `max_length` is compiled with XLA. The first run is expensive, but subsequent runs are much faster.

**Demo-3:** You can also provide batched prompts using a list as input:

In [21]:
start_time = time.perf_counter()

results = gemma_lm.generate(
    ["what is keras in 3 bullet points?",
     "list, in less than 100 words, 5 things you can help me do?"],
    max_length=128)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 105162.119 ms
['what is keras in 3 bullet points?\n\n* **High-level API:** Keras provides a user-friendly interface for building and training neural networks, making it accessible to both beginners and experts.\n* **Flexible and Extensible:** Keras is built on top of TensorFlow, offering a wide range of customization options and the ability to integrate with other libraries.\n* **Production-ready:** Keras models can be deployed and used in real-world applications, including image classification, object detection, and natural language processing. \n<end_of_turn>', 'list, in less than 100 words, 5 things you can help me do?\n\n1. **Write a poem about a lost cat.**\n2. **Create a list of fun activities for a rainy day.**\n3. **Generate a funny joke.**\n4. **Help me brainstorm ideas for a birthday party.**\n5. **Explain a complex scientific concept in simple terms.** \n\n\n<end_of_turn>']


**Demo-3 Imitation (Spanish)**

In [24]:
start_time = time.perf_counter()

results = gemma_lm.generate(
    ["El mejor gol de Di María en su carrera?",
     "lista, en menos de 250 palabras, Los 5 mejores goles de Di María en su carrera?"],
    max_length=256)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 239840.454 ms
['El mejor gol de Di María en su carrera?\n\nEl mejor gol de Di María en su carrera es un tema de opinión, ya que hay muchos que podrían considerarse como candidatos. \n\nSin embargo, algunos de los goles más populares y recordadas incluyen:\n\n* **Su golazo contra el Real Madrid en la Copa de la Liga de 2019.** Este gol, con un tiro cruzado desde la frontal del área, fue considerado un verdadero espectáculo de técnica y precisión.\n* **Su gol de tiro libre contra el Atlético Madrid en la temporada 2017-18.** Este gol, con una ejecución impecable, marcó el 3-0 y sentó la base de una victoria contundente.\n* **Su gol de chilena contra el Sevilla en la temporada 2016-17.** Este gol, con una ejecución magistral, fue un verdadero golazo y demostró la calidad y versatilidad técnica de Di María.\n* **Su gol de chilena contra el Bayern Munich en la temporada 2014-15.** Este gol, con una ejecución impresionante, fue uno de los goles más memorables del año.\n\nEn d

### Optional: Try a different sampler

You can control the generation strategy for `GemmaCausalLM` by setting the `sampler` argument on `compile()`. By default, [`"greedy"`](https://keras.io/api/keras_nlp/samplers/greedy_sampler/#greedysampler-class) sampling will be used.

**Demo-4:** As an experiment, try setting a [`"top_k"`](https://keras.io/api/keras_nlp/samplers/top_k_sampler/) strategy:

In [23]:
start_time = time.perf_counter()

gemma_lm.compile(sampler="top_k")
results = gemma_lm.generate("list, in less than 100 words, 5 things you can help me do?", max_length=128)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 33606.445 ms
list, in less than 100 words, 5 things you can help me do?

1. **Summarize a long text.**
2. **Translate languages.**
3. **Write different kinds of creative text formats.**
4. **Answer questions about a wide variety of topics.**
5. **Provide information about different cultures.** 


Let me know if you are up to the task!
<end_of_turn>


**Demo-4 Imitation (German)**

In [25]:
start_time = time.perf_counter()

gemma_lm.compile(sampler="top_k")
results = gemma_lm.generate("liste, in SST enthält 250 Wörter, Die 5 besten Spiele der Euro 2024?", max_length=256)

end_time = time.perf_counter() 
execution_time = (end_time - start_time) * 1000 
print(f"Inference Time: {execution_time:.3f} ms")

print(results)

Inference Time: 75627.002 ms
liste, in SST enthält 250 Wörter, Die 5 besten Spiele der Euro 2024?

**Top 5 Euro 2024-Spiele:**

Die Euro 2024 wird mit Spannung erwartet und es ist schwer, die Top 5-Spiele zu bestimmen, aber hier sind einige, die definitiv ein Highlight werden könnten:

1.  **Deutschland - England**: Die Klassiker der europäischen Fußballgeschichte treffen sich in der Gruppe! Diese Begegnung wird mit Spannung erwartet und ist eine klare Favoriten-Wahl mit einer starken deutschen Mannschaft.
2.  **Frankreich - Belgien**: Die beiden französischen Nachbarn liefern sich eine spannende Partie mit viel Emotionen! Frankreich hat die Qualifikation mit einer dominant-en Leistung erreicht und Belgien ist immer eine ernstzunehmende Gegnerin.
3.  **Niederlande - Italien**: Die Niederlande sind mit einem jungen und dynamischen Team eine der größten Überraschungen der Vergangenheit. Italy ist ein echter Herausforderer mit einer erfahrenen und starken Nationalmannschaft.
4.  **Spanien

While the default greedy algorithm always picks the token with the largest probability, the top-K algorithm randomly picks the next token from the tokens of top K probability.

You don't have to specify a sampler, and you can ignore the last code snippet if it's not helpful to your use case. If you'd like learn more about the available samplers, see [Samplers](https://keras.io/api/keras_nlp/samplers/).

## What's next

In this tutorial, you learned how to generate text using KerasNLP and Gemma. Here are a few suggestions for what to learn next:

* Learn how to [finetune a Gemma model](https://ai.google.dev/gemma/docs/lora_tuning).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn about [Gemma integration with Vertex AI](https://ai.google.dev/gemma/docs/integrations/vertex)
* Learn how to [use Gemma models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).